# CTC Forward, Backward and Gradient Practice. Compared with tf.ctc_loss.

Credit 是此篇 [DingKe ipynb](https://github.com/DingKe/ml-tutorial/blob/master/ctc/CTC.ipynb) 的, 他完整呈現了 CTC loss 以及 gradient 的計算, 非常棒!

此筆記加入自己的說明, 並且最後使用 tensorflow 來驗證.

這篇另一個主要目的為改成可以練習的格式 (**#TODO tag**). 因為我相信最好的學習方式是自己造一次輪子, 所以可以的話, 請試著完成把 #TODO tag 的部分做完吧.

我們只專注在 CTC loss 的 forward, backwark and gradient. Decoding 部分請參考原作者的 [ipynb](https://github.com/DingKe/ml-tutorial/blob/master/ctc/CTC.ipynb). 最後使用 tf.nn.ctc_loss and tf.gradients 與我們的計算做對比

In [0]:
import numpy as np
import tensorflow as tf

## 一些變數定義

1. `Vocab = [0, 1, 2, 3, 4]`, 其中 **0 表示 'blank'**.

2. `V = len(Vocab) = 5` 是字典大小, 譬如字典有 4 個 labels 加上一個 blank, 因此 V=5

3. `l` 是正確答案 (已經安插 blanks 了), 例如 `l = [0, 3, 0, 3, 0, 4, 0]`

4. `L = len(l) = 7`

5. **後驗概率 `y`** (shape=`[V,T]`), 其中 `T` 表示 input sequence 長度, 所以 `y[k,t]` 表示時間點 `t`, label `k` 的後驗概率


In [0]:
Vocab = [0,1,2,3,4]
l = [0, 3, 0, 3, 0, 4, 0]
V, L = len(Vocab), len(l)
T = 12
logits = np.random.random([V,T])

def softmax(logits):
    max_value = np.max(logits, axis=0, keepdims=True)
    exp = np.exp(logits - max_value)
    exp_sum = np.sum(exp, axis=0, keepdims=True)
    dist = exp / exp_sum
    return dist
  
y = softmax(logits)

## Forward/Backward Dynamic Programming

整個 CTC 關鍵就在計算 Dynamic Programming Table, 我們需要計算如下圖的 DP Table:
<img src="https://i.imgur.com/lOPaABD.png" height="70%" width="70%">


In [0]:
def forward(y, label):
  L = len(label)
  V, T = y.shape
  alpha = np.zeros([L,T])
  # init first column
  alpha[0,0] = # TODO
  alpha[1,0] = # TODO
  # run dp
  for t in range(1,T):
    for s in range(L):
      k = label[s]
      y_k_t = y[k,t]
      alpha_tmp = alpha[s,t-1]
      if s>0:
        alpha_tmp += # TODO
      if s>1 and k!=0 and k!=label[s-2]:
        alpha_tmp += # TODO
      alpha[s,t] = alpha_tmp*y_k_t
  return alpha

In [0]:
def backward(y,label):
  L = len(label)
  V, T = y.shape
  beta = np.zeros([L,T])
  # init last column
  beta[-1,-1] = # TODO
  beta[-2,-1] = # TODO
  # run dp
  for t in range(T-2,-1,-1):
    for s in range(L):
      k = label[s]
      y_k_t = y[k,t]
      beta_tmp = beta[s,t+1]
      if s<L-1:
        beta_tmp += # TODO
      if s<L-2 and k!=0 and k!=label[s+2]:
        beta_tmp += # TODO
      beta[s,t] = beta_tmp*y_k_t
  return beta

In [0]:
# Forward and Backward likelihood should be very close

alpha = forward(y,l)
likelihood_by_forward =  alpha[-1,-1] + alpha[-2,-1]
print('likelihood_by_forward = {}'.format(likelihood_by_forward))

beta = backward(y,l)
likelihood_by_backword =  beta[0,0] + beta[1,0]
print('likelihood_by_backword = {}'.format(likelihood_by_backword))

## Gradient w.r.t. Posterior $y$

[論文](https://www.cs.toronto.edu/~graves/icml_2006.pdf)的 (14) 如下:

$$
p(l|x)=\sum_{s=1}^{|l'|}\frac{\alpha_t(s)\beta_t(s)}{y_{l_s}^t}
$$

注意微分時 $\alpha_t(s)$ 與 $\beta_t(s)$ 是跟 $y_{l_s}^t$ 有關, 不能視為 constant, 利用它們的定義直接展開求微分即可. 譬如我們以 $t=2$ 當例子, 並且假設只有一條 alignment $\pi=\{\pi_0,\pi_1,...,\pi_T\}$, 其中 $y_{\pi_2}^2=y_{l'_s}^2$, 就是先忽略 $\sum_{\pi\in B^{-1}(l)}$, 不影響推導

$$
p(l|x)=\sum_{s=1}^{|l'|}\frac{\alpha_2(s)\beta_2(s)}{y_{l'_s}^2}\\
=\sum_{s=1}^{|l'|}\frac{y_{\pi_0}^0 y_{\pi_1}^1 (y_{l'_s}^2)^2 y_{\pi_3}^3 ... y_{\pi_T}^T}{y_{l'_s}^2}\\
=\sum_{s=1}^{|l'|} {y_{\pi_0}^0 y_{\pi_1}^1 (y_{l'_s}^2) y_{\pi_3}^3 ... y_{\pi_T}^T}\\
\frac{\partial p(l|x)}{\partial y_{l'_s}^2}={y_{\pi_0}^0 y_{\pi_1}^1 y_{\pi_3}^3 ... y_{\pi_T}^T} \mbox{,  只保留 index }s\\
=\frac{\alpha_2(s)\beta_2(s)}{(y_{l'_s}^2)^2}
$$

因此得到 Gradient:
$$
\frac{\partial p(l|x)}{\partial y_k^t}=\frac{1}{{y_k^t}^2}\sum_{s\in lab(l,k)}\alpha_t(s)\beta_t(s)
$$

解釋一下 $lab(l,k)$, 以我們上面的例子來說, `l = [0, 3, 0, 3, 0, 4, 0]`, `Vocabulary = [0,1,2,3,4]`, 而 $k$ 是指 Vocabulary 的哪一個值

- $lab(l,0)=\{0,2,4,6\}$
- $lab(l,1)=\{\}$
- $lab(l,2)=\{\}$
- $lab(l,3)=\{1,3\}$
- $lab(l,4)=\{5\}$

In [0]:
def lab_l_k(l,k):
  return ## TODO
print(lab_l_k(l,0))  # should be = [0, 2, 4, 6]
print(lab_l_k(l,1))  # should be = []
print(lab_l_k(l,3))  # should be = [1, 3]

因此我們的 gradient 計算公式如下
$$
\frac{\partial \ln p(l|x)}{\partial y_k^t}=\frac{1}{p(l|x)}\frac{\partial p(l|x)}{\partial y_k^t}
$$

In [0]:
def gradient(y, label):
  L = len(label)
  V, T = y.shape
  alpha = forward(y,label)
  beta = backward(y,label)
  grad = np.zeros([V,T])
  p = alpha[-1,-1] + alpha[-2,-1]
  for t in range(T):
    for k in range(V):
      for s in lab_l_k(label,k):
        grad[k,t] += # TODO
  grad /= (y*y*p)
  return grad

In [0]:
grad = gradient(y, l)
print(grad)
print(grad.shape)

用數值方法確認 gradient 計算, $\frac{df(x)}{dx}\simeq \frac{f(x+\delta)-f(x-\delta)}{2\delta}$, 如果我們只用 $\frac{df(x)}{dx}\simeq \frac{f(x+\delta)-f(x)}{\delta}$ 可以嗎? 是可以, 但是精確度較差, 原因可以由泰勒展開式知道:
$$
f(x+\delta)=f(x)+f'(x)\delta+\frac{1}{2}f''(x)\delta^2+o(|\delta|^3)\\
f(x-\delta)=f(x)-f'(x)\delta+\frac{1}{2}f''(x)\delta^2+o(|\delta|^3)\\
$$

比較兩種方式的計算結果:

$$
\frac{f(x+\delta)-f(x-\delta)}{2\delta}=f'(x)+o(|\delta|^2)\\
\frac{f(x+\delta)-f(x)}{\delta}=f'(x)+\frac{1}{2}f''(x)\delta+o(|\delta|^2)=f'(x)+o(|\delta|)
$$

可以發現 $\frac{df(x)}{dx}\simeq \frac{f(x+\delta)-f(x)}{\delta}$ 估計的 error 高出一個 order.

In [0]:
def check_grad(y, label, k=-1, t=-1, toleration=1e-3):
  grad_1 = gradient(y, label)[k, t]

  delta = 1e-10
  original = y[k, t]

  y[k, t] = original + delta
  alpha = forward(y, label)
  log_p1 = np.log(alpha[-1,-1] + alpha[-2,-1])

  y[k, t] = original - delta
  alpha = forward(y, label)
  log_p2 = np.log(alpha[-1,-1] + alpha[-2,-1])

  y[k, t] = original

  grad_2 = # TODO
  if np.abs(grad_1 - grad_2) > toleration:
    print('[%d, %d]：%.2e' % (k, t, np.abs(grad_1 - grad_2)))

for toleration in [1e-5, 1e-6]:
  print('%.e' % toleration)
  for w in range(y.shape[0]):
    for v in range(y.shape[1]):
      check_grad(y, l, w, v, toleration)

## Gradient w.r.t. Logits $u$

<img src="https://i.imgur.com/1k0g7N2.png" height="40%" width="40%">

如上圖, 我們已經計算了 $\frac{\partial L}{\partial y_i}$ (這邊假定固定的 $t$, 因此省略不寫), 我們希望計算 $\frac{\partial L}{\partial u_k}$, 由 chain rule 知道如下關係

$$
\frac{\partial L}{\partial u_k}=\sum_i^V\frac{\partial L}{\partial y_i}\frac{\partial y_i}{\partial u_k}
$$

而 softmax 的 gradient 如下:

$$
\frac{\partial y_i}{\partial u_k}=y_i(\delta_{ik}-y_k)
$$

所以結合起來變成:

$$
\frac{\partial L}{\partial u_k}=\sum_i^V\frac{\partial L}{\partial y_i}\frac{\partial y_i}{\partial u_k}\\
=\sum_i^V \left(\frac{1}{p(l|x){y_i}^2}\sum\alpha\beta\right)y_i(\delta_{ik}-y_k)\\
=\sum_i^V \left(\frac{\sum\alpha\beta}{p(l|x)y_i}\right)(\delta_{ik}-y_k)\\
=\sum_i^V \left(\frac{\sum\alpha\beta}{p(l|x)y_i}\right)\delta_{ik}-y_k\sum_i^V \left(\frac{\sum\alpha\beta}{p(l|x)y_i}\right)\\
=\frac{\sum\alpha\beta}{p(l|x)y_k}-y_k
$$ 

最後一行的第一個 term 是因為只有 $i=k$時 $\delta_{ik}=1$, 第二個 term 是因為 $\sum_i^V\frac{\sum\alpha\beta}{y_i}=p(l|x)$, 請參考論文 $\frac{\alpha\beta}{y_i}$的物理意義

總結來說, 將原來 gradient 計算:
$$
\frac{\partial L}{\partial y_i}=\frac{\sum\alpha\beta}{p(l|x){y_i}^2}
$$
改成如下計算:
$$
\frac{\partial L}{\partial u_k}=\frac{\sum\alpha\beta}{p(l|x)y_k}-y_k
$$

In [0]:
def gradient_logits_naive(y, label):
  '''
  gradient by back propagation
  '''
  y_grad = gradient(y, label)

  sum_y_grad = np.sum(y_grad * y, axis=0, keepdims=True)
  u_grad = y * (y_grad - sum_y_grad) 

  return u_grad

def gradient_logits(y, label):
  L = len(label)
  V, T = y.shape
  alpha = forward(y,label)
  beta = backward(y,label)
  grad = np.zeros([V,T])
  p = alpha[-1,-1] + alpha[-2,-1]
  for t in range(T):
    for k in range(V):
      for s in lab_l_k(label,k):
        grad[k,t] += # TODO
  grad /= (y*p)
  grad -= # TODO
  return grad

grad_l = gradient_logits_naive(logits, l)
grad_2 = gradient_logits(logits, l)

print(np.sum(np.abs(grad_l - grad_2)))

## Tensorflow Gradient w.r.t. Logits $u$

使用 tensorflow [`tf.nn.ctc_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/ctc_loss) 來驗證我們的計算

tf.nn.ctc_loss(
    **labels,
    inputs,
    sequence_length**,
    preprocess_collapse_repeated=False,
    ctc_merge_repeated=True,
    ignore_longer_outputs_than_inputs=False,
    time_major=True
)

很多參數, 我們就看主要的前三個
- `labels`: An int32 SparseTensor. labels.indices[i, :] == [b, t] means labels.values[i] stores the id for (batch b, time t). labels.values[i] must take on values in [0, **num_labels**). See core/ops/ctc_ops.cc for more details.
- `inputs`: 3-D float Tensor. If time_major == False, this will be a Tensor shaped: [batch_size, max_time, **num_classes**]. If time_major == True (default), this will be a Tensor shaped: [max_time, batch_size, **num_classes**]. The logits.
- `sequence_length`: 1-D int32 vector, size [batch_size]. The sequence lengths.


其中有四點要注意:
1. `labels` 用的是 num_labels, 而 `inputs` 用的是 num_classes, 它們的關係為: num_classes = num_labels + 1. 所以 num_labels 表示的是 true labels, 而 num_classes 是多了一個 'blank' 的數量
2. 'blank' 的 index 不像我們上面一樣用 0 表示, 會是 num_classes-1.
3. tf 的 ctc_loss 用的 input 直接就是 logits $u$, 不像我們上面算 gradient 時給的是後驗概率 $y$
4. tf 是 loss, 不是 likelihood, 所以算的 gradient 跟我們上面的正負號相反


#### 1. 建立 inputs

In [0]:
# logits.shape = y.shape = [V,T]
perm_idx = list(range(1,V))+[0]
inputs = logits[perm_idx,:].T.reshape([T,1,V]) # [max_time, batch_size, num_classes]
inputs_tensor = tf.constant(inputs, dtype=tf.float32)

#### 2. 建立 labels

In [0]:
# 原來 l = [0, 3, 0, 3, 0, 4, 0], 在這邊要變成 [4, 2, 4, 2, 4, 3, 4], true labels = {0,1,2,3}, blank={4}
# blank 不用自己填, 因此給 tensorflow 只需要 [2,2,3] 這樣的 array
labels_tensor = tf.SparseTensor(indices=[[0, 0], [0, 1], [0, 2]], values=[2, 2, 3], dense_shape=[1, 3])# 原來 l = [0, 3, 0, 3, 0, 4, 0], 在這邊要變成 [4, 2, 4, 2, 4, 3, 4], true labels = {0,1,2,3}, blank={4} (blank 不用自己填)
labels_tensor = tf.SparseTensor(indices=[[0, 1], [0, 3], [0, 5]], values=[2, 2, 3], dense_shape=[1, 7])

### 3. 建立 ctc_loss, 和計算 gradient

In [0]:
ctc_loss = tf.nn.ctc_loss(labels_tensor, inputs_tensor, [12])
print('inputs_tensor={}\nctc_loss={}'.format(inputs_tensor,ctc_loss))
tf_grad = tf.gradients(ys=ctc_loss,xs=inputs_tensor)
with tf.Session() as sess:
  tf_grad_results = sess.run(tf_grad)
print(np.array(tf_grad_results).shape)

### 4. 比較 tf 算的 gradient

In [0]:
# tf 是 loss, 我們之前算的是 likelihood, 因此差一個正負號
tf_grad_results = -np.array(tf_grad_results).reshape([T,V]).T

# tf 的 'blank' 是最大的 index, 將他挪回 index 0, true index 全部加 1
perm_idx = [V-1]+list(range(0,V-1))
tf_grad_results = tf_grad_results[perm_idx,:]

# print(tf_grad_results)
# print(np.array(tf_grad_results).shape)

# 我們之前計算的 gradient
grad = gradient_logits(y, l)
# print(grad)
# print(grad.shape)

# compare
diff_with_tf = np.mean(np.abs(tf_grad_results - grad))
print(diff_with_tf)

## Log-scale Forward/Backward/Gradient

 採用 Log scale 的前向後向以及 Gradient 計算會穩定很多

In [0]:
ninf = -np.float('inf')

def _logsumexp(a, b):
    '''
    np.log(np.exp(a) + np.exp(b))

    '''
    
    if a < b:
        a, b = b, a
        
    if b == ninf:
        return a
    else:
        return a + np.log(1 + np.exp(b - a)) 
    
def logsumexp(*args):
    '''
    from scipy.special import logsumexp
    logsumexp(args)
    '''
    res = args[0]
    for e in args[1:]:
        res = _logsumexp(res, e)
    return res

In [0]:
def forward_log(log_y, label):
  L = len(label)
  V, T = log_y.shape
  alpha = np.ones([L,T])*ninf
  # init first column
  alpha[0,0] = log_y[label[0],0]
  alpha[1,0] = log_y[label[1],0]
  # run dp
  for t in range(1,T):
    for s in range(L):
      ls = label[s]
      log_y_ls_t = log_y[ls,t]
      alpha_tmp = alpha[s,t-1]
      if s>0:
        alpha_tmp = # TODO
      if s>1 and ls!=0 and ls!=label[s-2]:
        alpha_tmp = # TODO
      alpha[s,t] = alpha_tmp + log_y_ls_t
  return alpha

log_alpha = forward_log(np.log(y), l)
alpha = forward(y, l)
print(np.sum(np.abs(np.exp(log_alpha) - alpha)))

In [0]:
def backward_log(log_y, label):
  L = len(label)
  V, T = log_y.shape
  beta = np.ones([L,T])*ninf
  # init last column
  beta[-1,-1] = log_y[label[-1],-1]
  beta[-2,-1] = log_y[label[-2],-1]
  # run dp
  for t in range(T-2,-1,-1):
    for s in range(L):
      ls = label[s]
      log_y_ls_t = log_y[ls,t]
      beta_tmp = beta[s,t+1]
      if s<L-1:
        beta_tmp = logsumexp(beta_tmp,beta[s+1,t+1])
      if s<L-2 and ls!=0 and ls!=label[s+2]:
        beta_tmp = logsumexp(beta_tmp,beta[s+2,t+1])
      beta[s,t] = beta_tmp + log_y_ls_t
  return beta

log_beta = backward_log(np.log(y), l)
beta = backward(y, l)
print(np.sum(np.abs(np.exp(log_beta) - beta)))

In [0]:
def gradient_log(log_y, label):
  L = len(label)
  V, T = log_y.shape
  log_alpha = forward_log(log_y,label)
  log_beta = backward_log(log_y,label)
  log_grad = np.ones([V,T])*ninf
  log_p = logsumexp(log_alpha[-1,-1], log_alpha[-2,-1])
  for t in range(T):
    for k in range(V):
      for s in lab_l_k(label,k):
        log_grad[k,t] = # TODO
  log_grad -= (2*log_y+log_p)
  return log_grad

log_grad = gradient_log(np.log(y), l)
grad = gradient(y, l)
print(np.sum(np.abs(np.exp(log_grad) - grad)))

Scaling 方法跳過

## Tensorflow ctc_decoder

使用上沒啥好說的, 有一點要注意的是如同上面說的 tf 會將最大的 label 視為 'blank', 所以下面 decode 會看到 label 0 但是沒有 label 4, 另外, 預設 merge_repeated=True 會去掉 blank 和 重複

tf.nn.ctc_greedy_decoder(
    inputs,
    sequence_length,
    merge_repeated=True
)

In [0]:
decoded, neg_sum_logits = tf.nn.ctc_greedy_decoder(inputs_tensor, [12])
with tf.Session() as sess:
  decoded_out, neg_sum_logits_out = sess.run([decoded, neg_sum_logits])
print(decoded_out)
print(neg_sum_logits_out)

tf.nn.ctc_beam_search_decoder(
    inputs,
    sequence_length,
    beam_width=100,
    top_paths=1,
    merge_repeated=True
)

In [0]:
decoded_beam, neg_sum_logits_beam = tf.nn.ctc_greedy_decoder(inputs_tensor, [12])
with tf.Session() as sess:
  decoded_beam_out, neg_sum_logits_beam_out = sess.run([decoded_beam, neg_sum_logits_beam])
print(decoded_beam_out)
print(neg_sum_logits_beam_out)

更多 decoding 的算法細節, 請參考 [DingKe](https://github.com/DingKe/ml-tutorial/blob/master/ctc/CTC.ipynb) 的文章